In [7]:
from langchain_community.document_loaders import PyPDFLoader


loader = PyPDFLoader(f"./高速公路养护管理办法.pdf")
pages = loader.load()

模型配置


In [1]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

OPENAI_API_KEY = 'FAKE_KEY'
OPENAI_BASE_URL = 'http://10.10.11.2:8001/v1'
OPENAI_MODEL_NAME = 'Qwen1.5-14B-Chat'

generator_llm = critic_llm  = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url=OPENAI_BASE_URL,
    model=OPENAI_MODEL_NAME,
)

# # generator with openai models
# generator_llm = ChatOpenAI(model="gpt-3.5-turbo")
# critic_llm = ChatOpenAI(model="gpt-3.5-turbo")

embeddings = OpenAIEmbeddings()

使用 ragas 生成 数据集，但是该生成的数据集没有 answer 字段。所以手动通过 gpt，输入 contexts 和 question 获取 answer。


In [10]:
generator = TestsetGenerator.from_langchain(
    generator_llm, # generates the questions
    critic_llm, # evolves the question to make it more relevant
    embeddings
)

distributions = {
    simple: 0.5,
    multi_context: 0.4,
    reasoning: 0.1,
}

language = "chinese"
generator.adapt(language, evolutions=[simple, reasoning, multi_context])

# generate testset
testset = generator.generate_with_langchain_docs(pages, test_size=10, distributions=distributions)
print(testset.to_dataset())

test_df = testset.to_pandas()
test_df.to_csv('./output2.csv', index=False, encoding='utf-8')

embedding nodes:   0%|          | 0/26 [00:00<?, ?it/s]

Filename and doc_id are the same for all nodes.


Generating:   0%|          | 0/10 [00:00<?, ?it/s]

Dataset({
    features: ['question', 'contexts', 'ground_truth', 'evolution_type', 'metadata', 'episode_done'],
    num_rows: 10
})


测试使用的数据集。使用 gpt-4 生成的。


In [4]:
from datasets import Dataset

obj = {
    "question": ["在遇到养护工程中的安全事故后，应如何按照应急预案进行报告？", 
                 "高速公路养护工程分为哪几类？",
                 "高速公路养护工程在安全管理中有哪些要求和责任追究机制？",
                 "各级管理机构如何确保高速公路养护工作的有效进行？",
                 "如何利用应急预案和信息化系统提升高速公路运维安全，同时优化隐患排查与决策执行效率？"
                 ],
    "answer": ["在遇到养护工程中的安全事故后，应按应急预案规定的程序及时报告集团及有关单位(或)政府部门。", 
               "根据第十三条，高速公路养护工程分为预防养护、修复养护、专项工程和应急养护四类。",
               "通过科学规划和效益分析，确保养护工程达标。采用预防性养护，强化动态管理、病害监测，及时发现并解决问题。",
               "养护分公司根据年度计划，制定季度和月度养护计划。",
               "各级管理机构需明确养护管理部门，配备足够专职人员，执行法规，制定管理制度，指导、检查、监督和考核养护工作，确保其有效进行。"
               ],
    "contexts": [['  - 6 -   理。事件发生后，应按应急预案规定的程序及时报告集团及有关\n单位（或）政府部门。  \n第二十二条  养护工程应加强综合管理，确保高速公路营运\n安全。因管理工作不到位而出现质量、安全事故，未能实现养护\n工作的质量、安全目标，视其性质严重程度追究有关单位和人员\n的责任。  \n第四章 前期工作  \n第二十三条  高速公路养护管理单位按照规范规定的检测指\n标和频率，定期组织对高速公路路基、路面、桥梁、隧道、附属\n设施等进行检测和评定。按照相关规范标准、国家和省高速公路\n养护规划、养护目标，分析养护需求，筛选排序拟实施养护工程\n项目并提出年度计划安排。在检查、检测、评定时如发现有危及\n高速公路营运安全的情况， 应及时将有关检测与评定资料上报集\n团，以加快决策速度，消除安全隐患。  \n第二十四条  高速公路养护管理单位应对拟实施养护工程的\n路段、构造物或者附属设施等，及时组织专项检测和方案论证，\n根据技术状况、病害情况及成因、发展趋势，综合考虑技术、经\n济、安全、环保等因素，按照全寿命周期综 合效益最佳的原则，\n合理确定技术方案。  \n第二十五条  高速公'],
        [" - 4 -   第三章 目标与一般要求  \n第十一条  养护管理目标，通过科学计划、合理组织、严谨\n实施与严格考核等养护生产活动，保持路况完好，确保高速公路\n安全、畅通、舒适、美观。  \n第十二条  养护工程目标：  \n(一)工程质量合格 率100%； \n(二)工程投资不突破批复投资计划 （含经批准调整后的投资\n计划）；  \n(三)坚决杜绝重大安全责任事故， 避免发生较大安全责任事\n故，控制一般安全责任事故 ； \n(四)工程基本建设程序完善，资料完整。   \n第十三条  养护工程分为预防养护、修复养护、专项工程和\n应急养护四类。  \n第十四条  髙速公路养护应加强技术管理，努力提高技术水\n平。积极推进专业化队伍、专业化设备、专业化技术，加大资金\n和技术投入，大力开展养护先进技术攻关，探索养护大数据平台\n建设、养护标准化基地建设，总结适应高寒地区养护工作的系统\n经验，促进养护实力提升 、积累养护工程业绩、提升养护 专业资\n质，适应更加开放的市场竞争。  \n第十五条  充分认识预防性养护的重要性和紧迫性， 贯彻 “预\n防为主、防治结合 ""的方针，加强动态管理和病害监测，根据养\n护质量检 "],
                 [" - 7 -   第五章 计划编制  \n第二十六条  高速公路的养护计划，按照作用和时间长短，\n分为中长期计划、年度计划、季度计划和月度计划：  \n（一）中长期计划：是高速公路经营期内的总体养护计划或\n阶段性养护计划，通常指三年及以上的养护计划。中长期计划的\n主要内容包括大中修规划、改造规划、养护资金的安排等。  \n（二）年度计划：指养护生产的当年实施计划。是按照中长\n期计划所确定的养护目标，根据高速公路现状，落实生产任务，\n有效地指导养护生产活动的计划。年度计划由养护分公司编制，\n高速公路养护管理单位审核。高速公路养护管理单位于每年的\n12月份将次年的年度计划报集团审批，当年的年度计划执行情\n况报集团备案。年度计划可在第三季度根据实际情况，经集团批\n准后进行调整。  \n（三）季度计划和月度计划： 养护分公司在年度计划的基础\n上，编制季度和月度养护计划。  \n第二十七条  养护工程计划编制应当优先安排以下项目：  \n（一）严重影响公众安全通行的；  \n（二）具有重大政治、经济和国防意义的；  \n（三）国家、省有明确 要求的事项；  \n（四）技术状况差、明显影响高速公路整体服务水 "],
                 [" - 6 -   理。事件发生后，应按应急预案规定的程序及时报告集团及有关\n单位（或）政府部门。  \n第二十二条  养护工程应加强综合管理，确保高速公路营运\n安全。因管理工作不到位而出现质量、安全事故，未能实现养护\n工作的质量、安全目标，视其性质严重程度追究有关单位和人员\n的责任。  \n第四章 前期工作  \n第二十三条  高速公路养护管理单位按照规范规定的检测指\n标和频率，定期组织对高速公路路基、路面、桥梁、隧道、附属\n设施等进行检测和评定。按照相关规范标准、国家和省高速公路\n养护规划、养护目标，分析养护需求，筛选排序拟实施养护工程\n项目并提出年度计划安排。在检查、检测、评定时如发现有危及\n高速公路营运安全的情况， 应及时将有关检测与评定资料上报集\n团，以加快决策速度，消除安全隐患。  \n第二十四条  高速公路养护管理单位应对拟实施养护工程的\n路段、构造物或者附属设施等，及时组织专项检测和方案论证，\n根据技术状况、病害情况及成因、发展趋势，综合考虑技术、经\n济、安全、环保等因素，按照全寿命周期综 合效益最佳的原则，\n合理确定技术方案。  \n第二十五条  高速公"],
                 ["  - 2 -   第七条 各级管理机构必须明确养护管理部门，合理配备足\n够的专职养护管理、技术人员，明确任职条件和职责。  \n第八条 集团负责宏观调控养护工作，主要职责如下：  \n（一）贯彻执行国家、行业主管部门制定的关于高速公路养\n护的法律、法规，制定本集团相应的养护管理制度和企业标准。  \n（二） 对高速公路养护管理单位的高速公路养护管理工作进\n行指导、检查、监督和考核。   \n（三） 确定养护工程总体规划， 审批年度养护工程生产计划。  \n（四）批复大中修施工图设计，主持修复、应急养护工程验\n收。 \n第九条 高速公路养护管理单位作为管理主体，在集团的授\n权下，负 责全集团养护工作的监督、指导、检查和管理。主要职\n责如下：  \n（一）贯彻执行国家、行业主管部门及集团制定的关于高速\n公路养护的法律、法规或管理制度，制定本 企业相应的各项养护\n管理办法和实施细则。  \n（二）对养护分公司的养护管理工作作出具体要求，并进行\n指导、检查、监督和考核。  \n（三）养护工程项目储备及计划管理。  \n（四）统一组织设计、施工、监理、技术咨询、养护设备的\n招标采购。  \n（五）审批小修工程施工图 ']，各级管理机构必须明确养护管理部门，合理配备足够的专职养护管理、技术人员，并明确任职条件和职责。他们需要执行相关法规，制定管理制度，进行指导、检查、监督和考核，以确保养护工作的有效进行。"],
                 ],
    "ground_truth": ["事件发生后，应按应急预案规定的程序及时报告集团及有关单位（或）政府部门。", 
                     "高速公路养护工程分为预防养护、修复养护、专项工程和应急养护四类。",
                     "高速公路养护工程应加强综合管理，确保营运安全。如果因管理工作不到位导致质量、安全事故，未能实现养护工作的质量、安全目标，将视其性质严重程度追究有关单位和人员的责任。",
                     "他们需要执行相关法规，制定管理制度，进行指导、检查、监督和考核，以确保养护工作的有效进行。",
                     "根据应急预案的规定，事件发生后应及时报告集团及有关单位或政府部门，以加快决策速度，消除安全隐患。同时，高速公路养护管理单位应充分依靠养护信息化管理系统，推进大数据与养护工作深度融合，利用检测数据进行评估，提高养护工程决策和管理能力。例如，通过系统发现危及营运安全的情况，可以及时上报集团，以优化隐患排查与决策执行效率。"
                    ]
}

dataset = Dataset.from_dict(obj)
dataset[0]

{'question': '在遇到养护工程中的安全事故后，应如何按照应急预案进行报告？',
 'answer': '在遇到养护工程中的安全事故后，应按应急预案规定的程序及时报告集团及有关单位(或)政府部门。',
 'contexts': ['  - 6 -   理。事件发生后，应按应急预案规定的程序及时报告集团及有关\n单位（或）政府部门。  \n第二十二条  养护工程应加强综合管理，确保高速公路营运\n安全。因管理工作不到位而出现质量、安全事故，未能实现养护\n工作的质量、安全目标，视其性质严重程度追究有关单位和人员\n的责任。  \n第四章 前期工作  \n第二十三条  高速公路养护管理单位按照规范规定的检测指\n标和频率，定期组织对高速公路路基、路面、桥梁、隧道、附属\n设施等进行检测和评定。按照相关规范标准、国家和省高速公路\n养护规划、养护目标，分析养护需求，筛选排序拟实施养护工程\n项目并提出年度计划安排。在检查、检测、评定时如发现有危及\n高速公路营运安全的情况， 应及时将有关检测与评定资料上报集\n团，以加快决策速度，消除安全隐患。  \n第二十四条  高速公路养护管理单位应对拟实施养护工程的\n路段、构造物或者附属设施等，及时组织专项检测和方案论证，\n根据技术状况、病害情况及成因、发展趋势，综合考虑技术、经\n济、安全、环保等因素，按照全寿命周期综 合效益最佳的原则，\n合理确定技术方案。  \n第二十五条  高速公'],
 'ground_truth': '事件发生后，应按应急预案规定的程序及时报告集团及有关单位（或）政府部门。'}

英文 prompt + 中文语料。LLM 使用默认的 openai


In [11]:
from ragas.metrics import faithfulness, context_relevancy, answer_relevancy, answer_correctness, context_recall
from ragas import evaluate

metrics = [faithfulness, context_relevancy, answer_relevancy, answer_correctness, context_recall]
score = evaluate(dataset, metrics=metrics)
print(score)
score.to_pandas()

Evaluating:   0%|          | 0/25 [00:00<?, ?it/s]

,question,answer,contexts,ground_truth,faithfulness,context_relevancy,answer_relevancy,answer_correctness,context_recall
0,在遇到养护工程中的安全事故后，应如何按照应急预案进行报告？,在遇到养护工程中的安全事故后，应按应急预案规定的程序及时报告集团及有关单位(或)政府部门。,[ - 6 - 理。事件发生后，应按应急预案规定的程序及时报告集团及有关\n单位（或）...,事件发生后，应按应急预案规定的程序及时报告集团及有关单位（或）政府部门。,1.0,0.000000,0.960942,0.736363,1.0
1,高速公路养护工程分为哪几类？,根据第十三条，高速公路养护工程分为预防养护、修复养护、专项工程和应急养护四类。,[ - 4 - 第三章 目标与一般要求 \n第十一条 养护管理目标，通过科学计划、合...,高速公路养护工程分为预防养护、修复养护、专项工程和应急养护四类。,1.0,0.043478,0.993256,0.995450,1.0
2,高速公路养护工程在安全管理中有哪些要求和责任追究机制？,通过科学规划和效益分析，确保养护工程达标。采用预防性养护，强化动态管理、病害监测，及时发现并...,[ - 7 - 第五章 计划编制 \n第二十六条 高速公路的养护计划，按照作用和时间...,高速公路养护工程应加强综合管理，确保营运安全。如果因管理工作不到位导致质量、安全事故，未能实...,1.0,0.000000,0.863221,0.719449,1.0
3,各级管理机构如何确保高速公路养护工作的有效进行？,养护分公司根据年度计划，制定季度和月度养护计划。,[ - 6 - 理。事件发生后，应按应急预案规定的程序及时报告集团及有关\n单位（或）政...,他们需要执行相关法规，制定管理制度，进行指导、检查、监督和考核，以确保养护工作的有效进行。,1.0,0.000000,0.838871,0.715269,1.0
4,如何利用应急预案和信息化系统提升高速公路运维安全，同时优化隐患排查与决策执行效率？,各级管理机构需明确养护管理部门，配备足够专职人员，执行法规，制定管理制度，指导、检查、监督和...,[ - 2 - 第七条 各级管理机构必须明确养护管理部门，合理配备足\n够的专职养护管...,根据应急预案的规定，事件发生后应及时报告集团及有关单位或政府部门，以加快决策速度，消除安全隐...,0.6,0.000000,0.820203,0.512419,1.0


设置 metrics 适应中文


In [8]:
from ragas.metrics import faithfulness, context_relevancy, answer_relevancy, answer_correctness, context_recall
from ragas import evaluate
from ragas import adapt
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url=OPENAI_BASE_URL,
    model=OPENAI_MODEL_NAME,
)

openai_model = ChatOpenAI(model_name="gpt-4")

In [12]:
adapt(metrics=[faithfulness, context_relevancy, answer_relevancy, answer_correctness, context_recall], language="中文", llm=llm)
print(context_recall.context_recall_prompt.to_string())

Given a context, and an answer, analyze each sentence in the answer and classify if the sentence can be attributed to the given context or not. Use only "Yes" (1) or "No" (0) as a binary classification. Output json with reason.

The output should be a well-formatted JSON instance that conforms to the JSON schema below.

As an example, for the schema {{"properties": {{"foo": {{"title": "Foo", "description": "a list of strings", "type": "array", "items": {{"type": "string"}}}}}}, "required": ["foo"]}}
the object {{"foo": ["bar", "baz"]}} is a well-formatted instance of the schema. The object {{"properties": {{"foo": ["bar", "baz"]}}}} is not well-formatted.

Here is the output JSON schema:
```
{{"type": "array", "items": {{"$ref": "#/definitions/ContextRecallClassificationAnswer"}}, "definitions": {{"ContextRecallClassificationAnswer": {{"title": "ContextRecallClassificationAnswer", "type": "object", "properties": {{"statement": {{"title": "Statement", "type": "string"}}, "attributed": {

中文 prompt + 中文数据集进行测试


In [13]:
metrics = [faithfulness, context_relevancy, answer_relevancy, answer_correctness, context_recall]

print(context_recall.context_recall_prompt.to_string()) # 检查prompt是否为中文
score = evaluate(dataset, metrics=metrics)
print(score)
score.to_pandas()

Given a context, and an answer, analyze each sentence in the answer and classify if the sentence can be attributed to the given context or not. Use only "Yes" (1) or "No" (0) as a binary classification. Output json with reason.

The output should be a well-formatted JSON instance that conforms to the JSON schema below.

As an example, for the schema {{"properties": {{"foo": {{"title": "Foo", "description": "a list of strings", "type": "array", "items": {{"type": "string"}}}}}}, "required": ["foo"]}}
the object {{"foo": ["bar", "baz"]}} is a well-formatted instance of the schema. The object {{"properties": {{"foo": ["bar", "baz"]}}}} is not well-formatted.

Here is the output JSON schema:
```
{{"type": "array", "items": {{"$ref": "#/definitions/ContextRecallClassificationAnswer"}}, "definitions": {{"ContextRecallClassificationAnswer": {{"title": "ContextRecallClassificationAnswer", "type": "object", "properties": {{"statement": {{"title": "Statement", "type": "string"}}, "attributed": {

Evaluating:   0%|          | 0/25 [00:00<?, ?it/s]

{'faithfulness': 0.9200, 'context_relevancy': 0.0087, 'answer_relevancy': 0.9006, 'answer_correctness': 0.7841, 'context_recall': 0.8333}


,question,answer,contexts,ground_truth,faithfulness,context_relevancy,answer_relevancy,answer_correctness,context_recall
0,在遇到养护工程中的安全事故后，应如何按照应急预案进行报告？,在遇到养护工程中的安全事故后，应按应急预案规定的程序及时报告集团及有关单位(或)政府部门。,[ - 6 - 理。事件发生后，应按应急预案规定的程序及时报告集团及有关\n单位（或）...,事件发生后，应按应急预案规定的程序及时报告集团及有关单位（或）政府部门。,1.0,0.000000,0.962356,0.986295,0.500000
1,高速公路养护工程分为哪几类？,根据第十三条，高速公路养护工程分为预防养护、修复养护、专项工程和应急养护四类。,[ - 4 - 第三章 目标与一般要求 \n第十一条 养护管理目标，通过科学计划、合...,高速公路养护工程分为预防养护、修复养护、专项工程和应急养护四类。,1.0,0.043478,1.000000,0.995453,1.000000
2,高速公路养护工程在安全管理中有哪些要求和责任追究机制？,通过科学规划和效益分析，确保养护工程达标。采用预防性养护，强化动态管理、病害监测，及时发现并...,[ - 7 - 第五章 计划编制 \n第二十六条 高速公路的养护计划，按照作用和时间...,高速公路养护工程应加强综合管理，确保营运安全。如果因管理工作不到位导致质量、安全事故，未能实...,1.0,0.000000,0.863197,0.594449,1.000000
3,各级管理机构如何确保高速公路养护工作的有效进行？,养护分公司根据年度计划，制定季度和月度养护计划。,[ - 6 - 理。事件发生后，应按应急预案规定的程序及时报告集团及有关\n单位（或）政...,他们需要执行相关法规，制定管理制度，进行指导、检查、监督和考核，以确保养护工作的有效进行。,1.0,0.000000,0.855797,0.465269,1.000000
4,如何利用应急预案和信息化系统提升高速公路运维安全，同时优化隐患排查与决策执行效率？,各级管理机构需明确养护管理部门，配备足够专职人员，执行法规，制定管理制度，指导、检查、监督和...,[ - 2 - 第七条 各级管理机构必须明确养护管理部门，合理配备足\n够的专职养护管...,根据应急预案的规定，事件发生后应及时报告集团及有关单位或政府部门，以加快决策速度，消除安全隐...,0.6,0.000000,0.821787,0.879085,0.666667
